# Moonboard  App
****

In [1]:
import requests
import bs4
import re
import tqdm
import pickle
import qgrid
from IPython.display import display
from ipywidgets import Button 
from IPython.display import HTML
import pandas as pd

MOONBOARD_PROBLEMS_URL = "http://www.moonboard.com/problems/"
problems = pickle.load(open('problems.pkl', 'rb'))

def get_all_problems_ids():
    """get all problems id"""
    problems={}
    r = requests.get(MOONBOARD_PROBLEMS_URL)
    w_soup = bs4.BeautifulSoup(r.content,'lxml')
    problems_tags = w_soup.find_all(lambda tag: tag.name == 'div' and ('problem-id' and 'grade-val' in tag.attrs))
    for p in problems_tags: 
        cl = p.get('class') 
        problems[p.get('problem-id')]= {'grade_val':p.get('grade-val'),
                                       'class':cl,
                                       'author':(" ".join(cl[4:])).strip()
                                       }                
    return problems

def get_new_problems_ids(old_problems=None):
    problems = get_all_problems_ids()
    new_problems_keys = set(problems.keys())- set(old_problems.keys())
    return {k:v for k,v in problems.items() if k in new_problems_keys}
    
    
def get_problem_data(problem_id):
    url = MOONBOARD_PROBLEMS_URL + "?p={}/".format(problem_id)
    r = requests.get(url)
    web_soup = bs4.BeautifulSoup(r.content,'lxml')
    summary= web_soup.findAll('div',attrs={'class':'summary'})
    if len(summary)>2:
        raise ValueError
    s = summary[1]
    p_info = {'name': s.find('h1',attrs={'class':'post-title'}).decode_contents().strip(),
              'grade':s.find('div',attrs={'id':'font_grade'}).decode_contents().strip().lower(),
              'hold_sets':[c.strip() for c in s.children if c.name!= 'div' and "Hold" in c ],
              'holds':{hold.get('id'):hold.get('name') for hold in s.find_all(id = re.compile("^FH|^IH|^SH"))}
             }#hold.get('id'):hold.get('name')
    return p_info

def validate_problem(problems_data):
    return True

def update_problems(problems, nmax = 10000):
    print('fetch problems ids')
    new_problems = get_new_problems_ids(problems)
    n=0
    errors=[]
    added=[]
    print('fetch single problems')
    for k,p in tqdm.tqdm_notebook(new_problems.items()):
        try:
            p_d = get_problem_data(k)
        except:
            errors.append(k)
        else:
            if validate_problem(p_d):
                problems[k] = {**p ,**p_d}
                added.append(k)
                n+=1
            else:
                errors.append(k)
        if n>nmax:
            break
    #save
    print("Save to file")
    with open('problems.pkl', 'wb') as output:
        # Pickle dictionary using protocol 0.
        pickle.dump(problems, output)
    print('=========\nTotal number of problems:' ,len(problems), '\nAdded:', added,'\nErrors:', errors)
    return problems,errors,added


## Fetch Problem from MOON website

at the following [url](http://www.moonboard.com/problems/) there is the full list of problems for the moonboard

In [2]:
update_button = Button()
update_button.description="Update Problems"

def update(b):
    update_problems(problems)

update_button.on_click(update)
display(update_button)

fetch problems ids
fetch single problems

Save to file
Total number of problems: 5162 
Added: ['127146', '127093', '127068', '126810', '127125', '127084', '127074', '127071', '126970', '127004', '127081', '126841', '127058', '126880', '127048', '127086', '126806'] 
Errors: ['89770', '61535']


In [3]:
import pandas as pd

d = {k:{key:v.get(key) for key in['name', 'grade', 'hold_sets','author' ] } for k,v in problems.items()}

sort = pd.DataFrame.from_dict(d , orient='index')


opts = {
    'fullWidthRows': True,
    'syncColumnCellResize': True,
    'forceFitColumns': True,
    'rowHeight': 28,
    'enableColumnReorder': False,
    'enableTextSelectionOnCells': True,
    'editable': False,
    'autoEdit': False
}
qgrid.show_grid(sort[['name', 'grade', 'hold_sets', 'author']], grid_options=opts)

## Drive Leds


****